wa# INFO 371 Problem Set #6: Naive Bayes

Chesie Yu

02/14/2022

<style type="text/css" >
  body{
    font-family: "Serif";
    font-size: 12pt
  }
  em {
	color: #4E7F9E;
  }
</style>

In [43]:
# Import the packages (as usual, the exercises...)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1 Load Data and Clean Data

1\. (2pt) Load and clean data. Feel free to copy-paste from your PS05 solution.  

In [2]:
# Load the data
email = pd.read_csv("lingspam-emails.csv.bz2", 
                    usecols = ["spam", "message"], sep = "\t")
email.head()

,spam,message
0,False,Subject: re : 2 . 882 s - > np np > date : su...
1,False,Subject: s - > np + np the discussion of s - ...
2,False,Subject: 2 . 882 s - > np np . . . for me it ...
3,False,"Subject: gent conference "" for the listserv ""..."
4,False,Subject: query : causatives in korean could a...


In [3]:
# Check the shape
email.shape

(2893, 2)

In [4]:
# Check for missings!
email.isna().sum()

spam       0
message    0
dtype: int64

In [5]:
# Check if there are any empty strings
np.sum(email.message == "")

0

_The data looks nice and clean (but with a lot of spams!)_

<br>

2\. (2pt) Vectorize emails so you have a DTM (I’ll refer to this as the design matrix $X$) and the spam/non-spam indicator $y$. If you don’t know how to do it, you can just use the code below:  

In [6]:
# Import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
# Set up the vectorizer 
vectorizer = CountVectorizer(binary = True)

# Create the DTM
X = vectorizer.fit_transform(email.message).toarray() # Learn vocabulary and return DTM
vocabulary = vectorizer.get_feature_names_out()       # Get output feature names for transformation
X = pd.DataFrame(X, columns = vocabulary)
X.head()

,00,000,0000,00001,00003000140,00003003958,00007,0001,00010,00014,...,zwischen,zwitserlood,zxgah7qabjh,zybatov,zybatow,zygmunt,zyokyoozyu,zytkow,zz214,zzlsa
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Define the output y as spam indicators
y = email["spam"] == True
y[:5]

0    False
1    False
2    False
3    False
4    False
Name: spam, dtype: bool

How many different documents (emails) and different tokens (words) do you have in this data?  

In [9]:
# Report the number of emails
email.shape[0]

2893

In [10]:
# Report the number of words
len(vocabulary)

60925

_There are **2893 documents** and **60925 tokens** in this data._

<br>

3\. (2pt) Split data into training/validation chunks.  

In [11]:
# Import train_test_split
from sklearn.model_selection import train_test_split

In [12]:
# Split DTM and email data into random training and validation subsets
Xt, Xv, yt, yv = train_test_split(X, y, test_size = 0.2)

<br>

4\. (2pt) Design a scheme to name your variables so you can understand from the variable name which mathematical concept it refers to. You need variables like  
- $Pr(S = 1)$: probability of spam  
- $Pr(S = 0|W = 1)$: probability the email is not spam given it cointains the word W  
- $log Pr(W = 1)$: log probability of word present  

Explain how would you name these examples.  

_The variable names will follow the scheme as displayed below:_ 
    
| Variable Name | Notation                 | Definition                                                          |
|---------------|--------------------------|--------------------------------------------------------------------|
| Pr_S1         | $$Pr(S = 1)$$            | Probability that the email is spam                               |
| Pr_S0         | $$Pr(S = 0)$$            | Probability that the email is non-spam                           |
| Pr_W1_S1      | $$Pr(W = 1\|S = 1)$$     | Conditional probability that the word is present in spam emails     |
| Pr_W1_S0      | $$Pr(W = 1\|S = 0)$$     | Conditional probability that the word is present in non-spam emails |
| logPr_S1      | $$log Pr(S = 1)$$        | Log probability that the email is spam                            |
| logPr_S0      | $$log Pr(S = 0)$$        | Log probability that the email is non-spam                        |
| logPr_W1_S1   | $$log Pr(W = 1\|S = 1)$$ | Log probability that the word is present in spam emails             |
| logPr_W1_S0   | $$log Pr(W = 1\|S = 0)$$ | Log probability that the word is present in non-spam emails         |
| L_S1          | $$\ell(S = 1\|W)$$       | Log-likelihood for spam                                            |
| L_S0          | $$\ell(S = 0\|W)$$       | Log-likelihood for non-spam                                        |

<br>
<br>

___

<br>

# 2 Naive Bayes

Before you get into the business, here are two warming-up exercises:

<br>

1\. (4pt) Here is a small excerpt from the initial DTM (before you split it into training/validation), corresponding to rows 946 to 948, and to columns 30,037..30,042:  
  
`  X[946:949, 30037:30042].toarray()`  
`  array([[0, 0, 0, 0, 0],`  
`         [0, 0, 1, 0, 0],`  
`         [0, 0, 0, 0, 0]])`  
  
What do these numbers show:   
Note: you should have the same numbers in your analysis, this is not random.

In [13]:
# Extract the excerpt from DTM
X.iloc[946:949, 30037:30042].values

array([[0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0]])

(a) which emails do the rows correspond to?  

In [14]:
# Find the corresponding emails
email.message[946:949]

946    Subject: efl position in israel  i have been a...
947    Subject: job : english in keio univ ( 2nd post...
948    Subject: icslp 96  = = = = = = = = = = = = = =...
Name: message, dtype: object

_The rows corresponds to **the emails at indices 946 to 948**, as shown above._

(b) Which words do the columns correspond to?   

In [15]:
# Find the corresponding words
X.columns[30037:30042]

Index(['interventions', 'intervient', 'interview', 'interviewed',
       'interviewing'],
      dtype='object')

_The columns correspond to the words **"interventions", "intervient", "interview", "interviewed", "interviewing"**._

(c) What does the single “1” in the middle of the table mean?  

_Since we are using the binary count vectorizer, the "1" in the middle indicates that **the presence of word "interview" in the email at row index 947**._

(d) What do the zeros mean?  

_The zeros mean that **the corresponding word does not appear in the corresponding email**._

<br>

2\. (2pt) What is the accuracy of the naive model that predicts all emails into the majority category?  

In [16]:
# Find the majority category
y.mode()

0    False
dtype: bool

In [17]:
# Predict all emails into the majority category
yhat = np.zeros(y.size)
yhat[:5]

array([0., 0., 0., 0., 0.])

In [18]:
# Compute the accuracy of this naive model
np.mean(y == yhat)

0.8337366055997235

_The accuracy of the naive model that predicts all emails into the majority category (which is non-spam) is **83.37%**._

<br>

Now the data is in shape and it is time to get serious. But first things first. Ensure you are familiar with Naive Bayes. Consult the readings, available on canvas.
- Lecture notes Section 7.3 is written with this task in mind.   
- Schutt & O’Neill is an easy and accessible (and long) introduction,  
- Whitten & Frank is a lot shorter but still accessible introduction.  

Good. Now you are ready with the preparatory work and it’s time to dive into the real thing. Let’s implement Naive Bayes. Use only training data in the fitting below. We also do it in a slightly simpler way and ignore the missing words when computing the probabilites, i.e. we only look for the effect of a word’s presence, not for the effect of its absence.

3\. (3pt) Compute the unconditional (log) probability that the email is spam/non-spam, $log Pr(S = 1)$, and $log Pr(S = 0)$. These probabilities are based on the values of $y$ (i.e. spam) alone. They do not contain information about the words in emails.

In [19]:
# Compute the log probabilities for spam/non-spam
logPr_S1 = np.log(np.mean(yt))
logPr_S0 = 1 - logPr_S1
logPr_S1, logPr_S0

(-1.798697918572976, 2.7986979185729757)

<br>

4\. (8pt) For each word $w$, compute the (log) probability that the word is present in spam emails, $log Pr(W = 1|S = 1)$, and (log) probability that the word is present in non-spam emails, $log Pr(W = 1|S = 0)$. These probabilities can easily be calculated from counts of how many times these words are present for each class.  
Hint: these computations are based on your BOW-s $X$. Look at ways to sum along columns in this matrix.  

In [20]:
# Compute the log probabilities for presence of word in spam/non-spam
logPr_W1_S1 = np.array([np.log(np.mean(Xt[w][yt == 1])) for w in Xt.columns])
logPr_W1_S0 = np.array([np.log(np.mean(Xt[w][yt == 0])) for w in Xt.columns])
logPr_W1_S1[:5], logPr_W1_S0[:5]

/tmp/ipykernel_2163/1974588754.py:2: RuntimeWarning: divide by zero encountered in log
  logPr_W1_S1 = np.array([np.log(np.mean(Xt[w][yt == 1])) for w in Xt.columns])
/tmp/ipykernel_2163/1974588754.py:3: RuntimeWarning: divide by zero encountered in log
  logPr_W1_S0 = np.array([np.log(np.mean(Xt[w][yt == 0])) for w in Xt.columns])


(array([-1.11175308, -1.1689115 , -5.94803499, -5.94803499,        -inf]),
 array([-1.7819681 , -3.00144509, -7.56579328,        -inf, -6.8726461 ]))

<br>

Now we are done with the estimator. Your fitted model is completely described by these four probability vectors: $log Pr(S = 1)$, $log Pr(S = 1)$, $log Pr(W = 1|S = 1)$, $log Pr(W = 1|S = 0)$. Let’s now pull out your validation data and do some predictions.  

5\. (10pt) For both classes, $S = 1$ and $S = 0$, compute the log-likelihood that the email belongs to this class. Log-likelihood is given as (7.3.13, page 260 for now) in lecture notes, and the equations on Schutt “Doing Data Science”, page 102.  
Computing the likelihoods involves sums of the previously computed probabilities, $log Pr(W = 1|S)$, and BOW elements $x_{ij}$. Start by doing this by whatever way you can get it done (e.g. loops). The most important thing is that you understand what you do!  
But if you want to write efficient code, use matrix product instead (it is ∼ 1000× faster than loops). You can also check out np.apply_along_axis.  

_The log-likelihoods can be computed using:_  
<em>$$\ell(S = 1|W) = log Pr(S = 1) + \sum^{K}_{j = 1} log Pr(W_j = 1|S = 1) \cdot \mathbb{1}(W_j = 1)$$</em>
<em>$$\ell(S = 0|W) = log Pr(S = 0) + \sum^{K}_{j = 1} log Pr(W_j = 1|S = 0) \cdot \mathbb{1}(W_j = 1)$$</em>

In [21]:
# Compute the log-likelihood for spam
L_S1 = logPr_S1 + Xv.values @ logPr_W1_S1
L_S1[:5]

/tmp/ipykernel_2163/572226100.py:2: RuntimeWarning: invalid value encountered in matmul
  L_S1 = logPr_S1 + Xv.values @ logPr_W1_S1


array([nan, nan, nan, nan, nan])

In [22]:
# Compute the log-likelihood for non-spam
L_S0 = logPr_S0 + Xv.values @ logPr_W1_S0
L_S0[:5]

/tmp/ipykernel_2163/3955572254.py:2: RuntimeWarning: invalid value encountered in matmul
  L_S0 = logPr_S0 + Xv.values @ logPr_W1_S0


array([nan, nan, nan, nan, nan])

<br>

6\. (2pt) How many log-likelihoods you have to compute? Explain why do you have to have this many log-likelihoods.  

In [23]:
L_S1.shape[0] + L_S0.shape[0]

1158

_We have to compute a total of **1158 log-likelihoods**, with 579 emails in our validation data and 2 log-likelihoods for each email to make predictions._

<br>

7\. (7pt) Based on the log-likelihoods, predict the class $S = 1$ or $S = 0$ for each email in the validation set.  

In [24]:
# Predict whether the email is spam based on log-likelihoods
yhat = L_S1 > L_S0
yhat[:5]

array([False, False, False, False, False])

<br>

8\. (3pt) Print the resulting confusion matrix and accuracy (feel free to use existing libraries).  

In [25]:
# Import the packages
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [26]:
# Print the confusion matrix
confusion_matrix(yv, yhat)

array([[481,   0],
       [ 98,   0]])

In [27]:
# Compute the accuracy
accuracy_score(yv, yhat)

0.8307426597582038

<br>

9\. (5pt) If your results are like mine, you can see that the results are not impressive at all, your model works no better than the naive guess. Explain why do you get such mediocre results.  
Note: just explain, but do not do anything about it! We’ll attack the problem in the next question with smoothing.  

_With missing words in our training sample, the log conditional probabilities may result in $log Pr(W = 1|S) = log(0) = -\infty$; this corrupts the model as we further multiply these log-probabilities with the indicators, which leads to "nan" in the resulting log-likelihoods. Then, if we further investigate the predictions yhat, we will see that the model categorizes every single email as non-spam as it fails to compare two "nan-s". So essentially it's performing the same task as the naive model that predicts all emails into the majority category in Question 2.2 - and in this sense, the accuracy doesn't really improve._

<br>
<br>

___

<br>

# 3 Add Smoothing

So, now you have your brand-new NB algorithm up and running. But the results are not impressive... As a next step, we add smoothing to the model.


<br>

1\. (2pt) As you will be doing validation below, your first task is to mold what you did above into two funcions: one for fitting and another one for predicting. You can mostly copy-paste your code from above.

In [ ]:
###### 正在施工中 ######

def fit(Xt, yt):
    # Compute the log probabilities for spam/non-spam
    logPr_S1 = np.log(np.mean(yt))
    logPr_S0 = 1 - logPr_S1
    logPr_S1, logPr_S0
    # Compute the log probabilities for presence of word in spam/non-spam
    logPr_W1_S1 = np.array([np.log(np.mean(Xt[w][yt == 1])) for w in Xt.columns])
    logPr_W1_S0 = np.array([np.log(np.mean(Xt[w][yt == 0])) for w in Xt.columns])
    logPr_W1_S1[:5], logPr_W1_S0[:5]
    
    
def predict(Xv, yv):
    # Compute the log-likelihood for spam
    L_S1 = logPr_S1 + Xv.values @ logPr_W1_S1
    L_S1[:5]
    # Compute the log-likelihood for non-spam
    L_S0 = logPr_S0 + Xv.values @ logPr_W1_S0
    L_S0[:5]
    # Predict whether the email is spam based on log-likelihoods
    yhat = L_S1 > L_S0
    yhat[:5]
    # Compute the accuracy
    accuracy_score(yv, yhat)

<br>

2\. (18pt) Add smoothing to the model. Smoothing amounts to assuming that we have “seen” every possible word $\alpha \geq 0$ times already, in both spam and non-spam emails. Note that $\alpha$ does not have to be an integer, and typically the best $\alpha < 1$.  
What you have to do is to re-compute the probabilities $log Pr(S)$, $log Pr(\bar{S})$, $log Pr(w|S)$, $log Pr(w|\bar{S})$, the predictions part will remain unchanged. So you should update your fitting function by adding an additional argument α to it, and modify the probabilities accordingly.
See Lecture notes, the subsection for smoothing; and Schutt p 103 and p 109 for more explanations.    
Note: it is advantageous to make your model as two functions: one for fitting, and another for prediction.  

<br>

3\. (5pt) Use your updated model for predictions with a few different $\alpha$-values and report the corresponding confusion matrix and accuracy.  
A well-implemented algorith should not spend more than a few seconds on both fitting and predicting. However, more important that you understand what you are doing!  

<br>

4\. (5pt) Find the best smoothing parameter $\alpha$.   
You can just run a loop over different values, but start with very small values ($10^{−8}$, $10^{−7}$ and such). Check out np.logspace about how to do such sequences. Use the largest $\alpha = 10$ or so.  
Note: this is fairly fast if your algorithm is fast. But even if your algorithm is slow, still do your best!  
If your results are like mine, your best accuracy will be > 99.5%. (But this result is random!)  

<br>

5\. (2pt) Plot how accuracy depends on $\alpha$. Use log-scale for $\alpha$!  

<br>
<br>

___

<br>

# 4 Interpretation

Naive Bayes is interpretable – in a little similar fashion like linear regression. But in only a little similar fashion. Namely, we can find the words that are the best predictors that an email is spam, and the best predictors that email is non-spam. And we want to look at reasonably common words only, say more frequent than 10 times in the data.

<br>

1\. (10pt) Which words are the best predictors that an email is spam? These are the word where $Pr(S = 1|W = 1)$ is large and $Pr(S = 0|W = 1)$ is small, or to put it differently, where $log Pr(S = 1|W = 1) − log Pr(S = 0|W = 1)$ is large.  
Explain why this is the case.  
Hint: you may re-check the concept of log-likelihood and how that is used for prediction.  

<br>

2\. (10pt) Find 10 best words to predict spam and 10 best words to predict non-spam. Comment your results.

<br>

___

How many hours did you spend on this PS?

<br>

# 5 Extra Credit: Implement Cross-Validation

Above we just did validation using testing/validation set approach. But cross-validation is better–it avoids much of the random noise when doing the split. Here your task is to implement CV yourself without using existing libraries!

- Implement $k$-fold CV. I recommend to implement it as a function that a) puts your data into random order; b) splits these into $k$ chunks; c) selects a chunk for testing and the others for training; d) trains your NB model on the training chunks; e) computes accuracy on training chunk; f) returns mean accuracy over all these $k$ trials. The function should also take $\alpha$ as an argument, this is the hyperparameter you are going to optimize.  
-  Find the optimal $\alpha$ by 5-fold CV using your own CV code. You have to find the cross-validated accuracies for a number of $\alpha$-s between 0 and 1. Present the accuracy as a function of $\alpha$ on a plot and indicate which one is the best $\alpha$.

In [35]:
import numpy
import watermark.watermark as watermark


watermark(iversions=True, globals_=globals())

'numpy     : 1.20.3\npandas    : 1.3.4\nmatplotlib: 3.4.3\n'

In [39]:
%load_ext watermark


The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


UsageError: unrecognized arguments: pandas


In [40]:
 %reload_ext watermark

In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [47]:
%watermark -v -m -p np,pd,plt

Python implementation: CPython
Python version       : 3.9.7
IPython version      : 7.29.0

np : not installed
pd : not installed
plt: not installed

Compiler    : GCC 9.4.0
OS          : Linux
Release     : 5.4.144+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 4
Architecture: 64bit

